In [ ]:
from dash import Dash, html, dcc, callback, Output, Input, State, no_update
from pysnmp.hlapi.v3arch.asyncio import *
import dash_cytoscape as cyto
import asyncio
import re
# enable svg export
cyto.load_extra_layouts()

HOST_IP = '127.0.0.1'
STRING = 'public'

async def get_oid(host,string,oid,snmpEng):
    results = [];
    iterator = await getCmd(
        snmpEng,
        CommunityData(string, mpModel=1),
        await UdpTransportTarget.create((host, 161)),
        ContextData(),
        ObjectType(ObjectIdentity(oid))
    )
    errorIndication, errorStatus, errorIndex, varBinds = iterator
    if errorIndication:
        print(errorIndication)
    elif errorStatus:
        print(
            "{} at {}".format(
                errorStatus.prettyPrint(),
                errorIndex and varBinds[int(errorIndex) - 1][0] or "?",
            )
        )
    else:
        results =  varBinds
    return results

async def get_oids(host,string,oidData):
    snmpDispatcher = SnmpEngine()
    results = []
    first_oid = ''
    while True:
        errorIndication, errorStatus, errorIndex, varBindTable = await bulkCmd(
            snmpDispatcher,
            CommunityData(string, mpModel=1),
            await UdpTransportTarget.create((host, 161)),
            ContextData(),
            0,50,
            *oidData,
            lookupMib=False,
            lexicographicMode=True
        )
        if errorIndication:
            print(f"ERROR INDICATION :{errorIndication}")
            break
        elif errorStatus:
            print(
                f"ERROR STATUS {errorStatus.prettyPrint()} at {varBinds[int(errorIndex) - 1][0] if errorIndex else '?'}"
            )
        else:
            for varBind in varBindTable:
                if first_oid == varBind[0]:
                    break
                results.append({'oid' : f"{varBind[0]}", 'value': f"{varBind[1].prettyPrint()}"})
        if first_oid == varBindTable[0][0]:
            break
        else:
            first_oid = varBindTable[0][0]
    return results

oid = [ObjectType(ObjectIdentity('1.0.8802.1.1.2.1.4.1'))]

links = []
#List of IP in the network that have LLDP and SNMP configured
device_list = [ ]
snmpEngine = SnmpEngine()

async def get_sysnames(dev_list):
    print(dev_list)
    runs = []
    async with asyncio.TaskGroup() as tg:
        for i,d in enumerate(dev_list):
            print(d)
            runs.insert(i, tg.create_task(get_oid(d,'public','1.3.6.1.2.1.1.5.0',snmpEngine)))
    return runs

async def get_data(dev_list,devices,elements):
    for i,d in enumerate(dev_list):
        print(d)
        for device in devices:
            if device['address'] == d:
                sysname = device['sysname']
                break
        v = await get_oids(d,'public',oid)
        idx_list = []
        idx_dicts = []
        for line in v:
            idx = re.match(r'1\.0\.8802\.1\.1\.2\.1\.4\.1\.1\.(\d{,3})\.\d*.(\d*)', line['oid'])
            if idx:
                if idx.group(2) not in idx_list:
                    idx_dicts.append({'idx': idx.group(2)})
                    idx_list.append(idx.group(2))
                if idx.group(1) == '9':
                    for intf in idx_dicts:
                        if intf['idx'] == idx.group(2):
                            intf['rem_sysname'] = line['value']
    
        for datapoint in idx_dicts:
            if (datapoint['rem_sysname'] != ''):
                elements.append({'data' : {'source': sysname, 'target' : datapoint['rem_sysname']}})
    return elements

async def main():
    dev_list = []
    elements = []
    runs = await get_sysnames(device_list)
    for i,r in enumerate(runs):
        res = r.result()
        print(res)
        sysname = str(res[0]).split(' = ')[1]
        sysname = sysname.split('.')[0]
        dev_list.append({'address': device_list[i], 'sysname' : str(res[0]).split(' = ')[1]})
        elements.append({'data' : {'id': str(res[0]).split(' = ')[1], 'label' : sysname + "(" + device_list[i]+")"}})
    elements = await get_data(device_list,dev_list,elements)
    app = Dash(__name__)
    app.layout = html.Div(style={'backgroundColor': '#111111','width': '100%', 'height': '100%'}, children=[
        html.H1("Network LLDP:", style={'textAlign': 'center','color': '#7FDBFF'}),
        
        html.H3("Graph type: ", style={'textAlign': 'left',"float": "left", 'color': '#7FDBFF'}),
        dcc.Dropdown(
            id='dropdown-update-layout',
            value='cose',
            #style={'width': '40%','margin-left': '30%'},
            style={'width': '15em',"float": "left", 'margin-top': '5px'},
            clearable=False,
            options=[
                {'label': name.capitalize(), 'value': name}
                for name in ['cose','grid', 'breadthfirst', 'random', 'preset','circle',  'concentric','cose-bilkent','spread','cola','euler','dagre','klay']
            ]
        ),
        html.Br(),html.Br(),html.Br(),
        html.Button("Download Image", id="btn-download",n_clicks=0,style={'width': '15em', 'height' : '2em',"align": "left"}),
        cyto.Cytoscape(
            id='lldp_topo',
            elements=elements,
            layout={'name': 'cose'},
            style={'width': '100%', 'height': '30em'},
            stylesheet=[
            {
                'selector': 'node',
                'style': {
                    'label': 'data(label)',
                    'background-color': 'blue',
                    'color': '#7FDBFF'
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'curve-style': 'segments',
                    'line-color': 'green'
                }
            }
            ]
        )
    ])
    @callback(Output('lldp_topo', 'layout'),
              Input('dropdown-update-layout', 'value'))
    def update_layout(layout):
        return {
            'name': layout,
            'animate': True
    }
    @callback(
        Output("lldp_topo", "generateImage"),
        Input("btn-download", "n_clicks")
    )
    def get_image(btn_download):
        if btn_download == 0:
            return no_update
        action = 'download'
        return {
            # File type to output of 'svg, 'png', 'jpg', or 'jpeg' (alias of 'jpg')
            'type': 'jpeg',
            'filename': 'lldp_topo',
            'action': action
            }
    app.run_server(debug=True)

await main()